_________
뉴스 요약 및 감정평가 해보기 성공! 내일 리스트로 만들어서 채용공고처럼 넣을 수 있는 꼴로 만들어줘서 넣어보구 모듈화진행

In [1]:
import sys

sys.path.append('/app/EZPZ_Project/modules/crawlers/news') #뉴스 크롤러
sys.path.append('/app/EZPZ_Project/modules/torchmodules') # 토치 모델 뉴슈 요약 및 감정평가 가져오기

In [2]:
#from daum_news_crawler import get_news
import daum_news_crawler
import naver_news_crawler
import news_crawlers

In [6]:
#모델이 필요로 하는라이브러리
!pip install nltk
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/1a/70/e63223f8116931d365993d4a6b7ef653a4d920b41d03de7c59499962821f/click-8.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 9.8 MB/s eta 0:00:00


In [3]:
from service_models import ServiceModels 
from finbertkr import FinBertKR
from mt5sum import MT5Sum
from t5basesum import T5BaseSum

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
"""#뉴스 요약 nltk 설치하기
import service_models
import mt5sum #뉴스 요약
import finbertkr # 뉴스 평가i
import t5basesum #리뷰 여기선 안씀"""

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/app/EZPZ_Project/scheduler/news_crawler_server/\삼성전자_daum.csv', encoding='utf-8')
# 뉴스기사 데스크탑에 new_file 폴더에 빼놨음 필요할 때 확인

In [6]:
df.tail() #뉴스 기사 잘 받았나 확인

,news_url,news_cont,pub_date
15,http://v.daum.net/v/20230815030101695,정부는 14일 한덕수 국무총리 주재로 국무회의를 열고 경제인 등 2176명을 15일...,20230815
16,http://v.daum.net/v/20230814163150766,[이데일리 김인경 기자] ‘하반기부터 반도체 업황도 개선된다더니….’삼성전자 주가 ...,20230814
17,http://v.daum.net/v/20230814183054183,(서울=연합뉴스) 김아람 기자 = 글로벌 경기침체에 따른 수요 위축 여파로 올해 상...,20230814
18,http://v.daum.net/v/20230815001329338,국내 주요 기업인 중 올 상반기 보수를 가장 많이 받은 기업인 신동빈 롯데그룹 회장...,20230815
19,http://v.daum.net/v/20230814174946001,(서울=뉴스1) 김민성 기자 = 삼성전자가 올해 상반기 시설투자에 약 25조원을 쏟...,20230814


In [7]:
one_data=df['news_cont'][0]
one_data #나중에 이걸 반복문 돌려서 각각 데이터 얻은것들 채용공고 uid 붙여주듯이 데이터 테이블에 추가해주면 됨 오케이?


'이차전지에 집중 투자했던 개인 투자자가 이달부터는 삼성전자를 쓸어 담는 ‘큰 손’으로 돌아왔다. 올해 국내 증시는 개인 투자자의 수급으로 주도 섹터가 결정된 만큼 시장 안팎에서는 개인 투자자의 수급 방향에 주목하고 있다.14일 한국거래소에 따르면 이달 들어(1~11일) 개인 투자자 순매수 상위 1위 종목은 삼성전자인 것으로 집계됐다. 개인 투자자는 이 기간 삼성전자 주식 6900억원어치를 순매수했다. 포스코홀딩스가 5875억원 규모 순매수로 그 뒤를 이었다. 기아(3위, 2895억원)와 LG전자(6위, 1575억원) 네이버(10위, 1500억원) 등도 이름을 올리며 이차전지 관련 종목 일색이었던 순매수 상위 종목이 다양해졌다. 개인 투자자의 적극적인 매수로 주가가 올해만 933.64%나 오른 에코프로(12위, 1283억원)은 10위권 밖으로 밀려났다.코로나19 팬데믹 시절 ‘국민주’로 불렸던 삼성전자는 올해 들어 개인 투자자가 가장 많이 팔아 치운 종목이다. 개인 투자자는 올해 들어 7월까지 총 10조5818억원 규모의 삼성전자 주식을 순매도했다. 하지만 이달 들어서는 외국인(701억원)과 기관(-8090억원)을 제치고 삼성전자를 가장 많이 사들인 수급 주체가 됐다.최근 들어 이차전지 관련주 주가 상승세가 주춤한 데다, 하반기 반도체 업황이 개선될 것이라는 전망이 나온 데 따른 것으로 분석된다. 증권가에서는 이달 들어 삼성전자 목표가를 9만~9만5000원을 제시하며 하반기부터 실적이 개선될 것이라는 전망을 하고 있다.가장 최근 관련 보고서를 낸 박유악 키움증권 연구원은 삼성전자 3분기 매출액이 전 분기 대비 11% 상승한 66조9000억원, 영업이익은 지난 2분기보다 653% 늘어난 5조원을 각각 기록할 것으로 봤다. 박 연구원은 “반도체 디램(DRAM)에서 영업이익이 흑자로 전환될 것”이라며 “HBM(고대역폭메모리) 등 기술 경쟁력도 재부각돼 하반기 주가 강세가 예상된다”고 말했다. HBM은 디램을 수직으로 연결해 데이터 처리 속도를 높인 차세대 디램 제품

In [8]:
one_data_check= ServiceModels() #모델 서빙 모듈 객체

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
one_sum=one_data_check.get_summary(one_data, 'news') #뉴스 요약 된다!!!

In [13]:
one_sum

'신종 코로나바이러스 감염증(코رونا) 팬데믹 시절 ‘국민주’로 불렸던 삼성전자가 이달부터 “큰 손”으로 돌아왔다.'

In [12]:
one_data_check.get_sentiment(one_sum) #무조건 뉴스 요약 후에 그 요약데이터를 감정에 넣어야 결과나옴

'neutral'